In [118]:
!pip install --upgrade numpy

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.1.1 requires numpy<2.0,>=1.16; python_version <= "3.11", but you have numpy 2.0.0 which is incompatible.
gradio 4.18.0 requires numpy~=1.0, but you have numpy 2.0.0 which is incompatible.
pyarrow 15.0.0 requires numpy<2,>=1.16.6, but you have numpy 2.0.0 which is incompatible.
scikit-learn 1.4.0 requires numpy<2.0,>=1.19.5, but you have numpy 2.0.0 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.0.0 which is incompatible.
shapely 2.0.3 requires numpy<2,>=1.14, but you have numpy 2.0.0 which is incompatible.
streamlit 1.31.0 requires numpy<2,>=1.19.3, but you have numpy 2.0.0 which is incompatible.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 2.0.0 which is incompatible.
tens


     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
    --------------------------------------- 0.2/16.5 MB 2.8 MB/s eta 0:00:06
    --------------------------------------- 0.4/16.5 MB 2.8 MB/s eta 0:00:06
   - -------------------------------------- 0.5/16.5 MB 2.7 MB/s eta 0:00:06
   - -------------------------------------- 0.7/16.5 MB 2.8 MB/s eta 0:00:06
   - -------------------------------------- 0.8/16.5 MB 2.9 MB/s eta 0:00:06
   -- ------------------------------------- 1.0/16.5 MB 2.7 MB/s eta 0:00:06
   -- ------------------------------------- 1.1/16.5 MB 2.9 MB/s eta 0:00:06
   --- ------------------------------------ 1.3/16.5 MB 2.9 MB/s eta 0:00:06
   --- ------------------------------------ 1.5/16.5 MB 3.1 MB/s eta 0:00:05
   --- -------

In [119]:
#venv toxic
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('train_data.csv')
df.sample(3)

,id,qid1,qid2,question1,question2,is_duplicate
37388,37388,68043,68044,How is everyone earning millions from home by ...,How are people earning billions from home by s...,1
302200,302200,1749,17057,How can changing 500 and 1000 rupee notes end ...,How can discontinuing 500 and 1000 rupee will ...,1
51946,51946,92064,92065,Why did my Instagram log me out?,Is it normal for Instagram to randomly log out...,1


In [3]:
new_df = df.sample(20000,random_state=2)

In [4]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
      "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
     "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
     "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
     # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q
    

In [5]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

In [6]:
new_df['q1_len'] = new_df['question1'].str.len() 
new_df['q2_len'] = new_df['question2'].str.len()
new_df['q1_num_words'] = new_df['question1'].apply(lambda row: len(row.split(" ")))
new_df['q2_num_words'] = new_df['question2'].apply(lambda row: len(row.split(" ")))

In [7]:
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return len(w1 & w2)
new_df['word_common'] = new_df.apply(common_words, axis=1)

In [8]:
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return (len(w1) + len(w2))
new_df['word_total'] = new_df.apply(total_words, axis=1)

In [9]:
new_df['word_share'] = round(new_df['word_common']/new_df['word_total'],2)

#### Advanced features

In [10]:
from nltk.corpus import stopwords

def fetch_token_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features

In [11]:
token_features = new_df.apply(fetch_token_features, axis=1)

new_df["cwc_min"]       = list(map(lambda x: x[0], token_features))
new_df["cwc_max"]       = list(map(lambda x: x[1], token_features))
new_df["csc_min"]       = list(map(lambda x: x[2], token_features))
new_df["csc_max"]       = list(map(lambda x: x[3], token_features))
new_df["ctc_min"]       = list(map(lambda x: x[4], token_features))
new_df["ctc_max"]       = list(map(lambda x: x[5], token_features))
new_df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
new_df["first_word_eq"] = list(map(lambda x: x[7], token_features))

In [12]:
import nltk
from nltk.metrics import edit_distance

def fetch_length_features(row):
    q1 = row['question1']
    q2 = row['question2']
    
    length_features = [0.0] * 3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    # Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens)) / 2
    
    # Longest common substring using edit distance
    lcs_length = len(q1) + len(q2) - 2 * edit_distance(q1, q2)
    length_features[2] = lcs_length / (min(len(q1), len(q2)) + 1)
    
    return length_features

length_features = new_df.apply(fetch_length_features, axis=1)

new_df['abs_len_diff'] = list(map(lambda x: x[0], length_features))
new_df['mean_len'] = list(map(lambda x: x[1], length_features))
new_df['longest_substr_ratio'] = list(map(lambda x: x[2], length_features))


In [13]:
# Fuzzy Features
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [14]:
fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

# Creating new feature columns for fuzzy features
new_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
new_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
new_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
new_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [15]:
print(new_df.shape)
new_df.head(2)

(20000, 28)


,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,...,0.923070,1,1,0,13.0,1.960526,99,99,99,99
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,48,56,13,16,...,0.466664,1,1,3,13.5,1.142857,69,67,67,74


In [16]:
from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(new_df[['cwc_min', 'cwc_max', 'csc_min', 'csc_max' , 'ctc_min' , 'ctc_max' , 'last_word_eq', 'first_word_eq' , 'abs_len_diff' , 'mean_len' , 'token_set_ratio' , 'token_sort_ratio' ,  'fuzz_ratio' , 'fuzz_partial_ratio' , 'longest_substr_ratio']])
y = new_df['is_duplicate'].values

In [68]:
ques_df = new_df[['question1','question2']]
ques_df.sample(5)

,question1,question2
153494,do you ask your ex why your relationship did n...,how have you dealt with losing love for your p...
52151,am i eligible to do a bmm after my diploma,i have a diploma in management am i eligible ...
123668,can terrorists not use snapchat,are terrorists using snapchat
105507,does the indian railway still operate non ac ...,what are the classes of travel in indian railway
153159,if you accidentally like someone s photo on fa...,if i like someone s picture in facebook and th...


In [82]:
ques_df.iloc[1500][0]

'why does a video message sent from the space ship to the earth takes 19 yrs to reach and 57 yrs to to get reply as seen in the movie passengers'

In [64]:
final_df = new_df.drop(columns=['id','qid1','qid2','question1','question2'])
print(final_df.shape)

(20000, 23)


In [19]:
final_df.head(2)

,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
398782,1,75,76,13,13,12,26,0.46,0.874989,0.874989,...,0.923070,1,1,0,13.0,1.960526,99,99,99,99
115086,0,48,56,13,16,8,24,0.33,0.666644,0.499988,...,0.466664,1,1,3,13.5,1.142857,69,67,67,74


##### Text vectorization using Word2Vec

In [86]:
from gensim.models import Word2Vec
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

tokenized_questions = [q.split() for q in questions]
#training skip-gram model 
w2v_model = Word2Vec(tokenized_questions, vector_size=100, window=5, min_count=1,sg=1) 
#Contextual Understanding: Since duplicate questions often require understanding the 
# context in which words are used, the skip-gram model's ability to better capture word semantics might provide an edge.

In [85]:
tokenized_questions

[['what',
  'is',
  'the',
  'best',
  'marketing',
  'automation',
  'tool',
  'for',
  'small',
  'and',
  'mid',
  'size',
  'companies'],
 ['i',
  'am',
  'poor',
  'but',
  'i',
  'want',
  'to',
  'invest',
  'what',
  'should',
  'i',
  'do'],
 ['i',
  'am',
  'from',
  'india',
  'and',
  'live',
  'abroad',
  'i',
  'met',
  'a',
  'guy',
  'from',
  'france',
  'in',
  'a',
  'party',
  'i',
  'want',
  'to',
  'date',
  'him',
  'how',
  'do',
  'i',
  'do',
  'that'],
 ['why',
  'do',
  'so',
  'many',
  'people',
  'in',
  'the',
  'u',
  's',
  'hate',
  'the',
  'southern',
  'states'],
 ['consequences', 'of', 'bhopal', 'gas', 'tragedy'],
 ['i',
  'killed',
  'a',
  'snake',
  'on',
  'a',
  'friday',
  'there',
  'is',
  'a',
  'belief',
  'that',
  'when',
  'you',
  'kill',
  'a',
  'snake',
  'on',
  'a',
  'friday',
  'it',
  'will',
  'certainly',
  'take',
  'revenge',
  'will',
  'i',
  'be',
  'killed'],
 ['is',
  'the',
  'royal',
  'family',
  'a',
  'net',
  

In [56]:
questions[2000]

'why are mobile plans for 28 days'

In [88]:
w2v_model.wv.get_normed_vectors().shape  # there are 21576 unique words with 100D vector representation

(21576, 100)

In [87]:
y = w2v_model.wv.index_to_key  # fetches you the words corresponding to vector representation
y

['the',
 'what',
 'is',
 'i',
 'how',
 'a',
 'to',
 'in',
 'do',
 'of',
 'are',
 'and',
 'can',
 'for',
 'you',
 'why',
 'it',
 'my',
 'best',
 'on',
 'does',
 'have',
 'if',
 'be',
 'or',
 'not',
 'which',
 'some',
 'that',
 'with',
 'should',
 'get',
 'an',
 'your',
 'from',
 'india',
 'will',
 's',
 'who',
 'when',
 'at',
 'people',
 'would',
 'good',
 'like',
 'as',
 'there',
 'about',
 'between',
 'one',
 'we',
 'most',
 'make',
 'am',
 'quora',
 'way',
 'did',
 'was',
 'by',
 'where',
 'after',
 'life',
 'any',
 'they',
 'me',
 'so',
 'money',
 'time',
 'know',
 'this',
 'someone',
 'has',
 'new',
 'difference',
 'learn',
 'find',
 'think',
 'much',
 'their',
 'use',
 'many',
 'more',
 'indian',
 'work',
 'all',
 'trump',
 'than',
 'ever',
 'without',
 'first',
 'but',
 'start',
 'mean',
 'world',
 'out',
 'online',
 'become',
 'other',
 '2',
 'year',
 'want',
 'better',
 'could',
 'love',
 'us',
 'job',
 'feel',
 'he',
 'into',
 'up',
 'possible',
 'take',
 'go',
 'day',
 'engli

In [89]:
# Most frequent words
most_freq_words = w2v_model.wv.index_to_key[:10]
print(f"Most frequent words: {most_freq_words}")

# Least frequent words
least_freq_words = w2v_model.wv.index_to_key[-10:]
print(f"Least frequent words: {least_freq_words}")


Most frequent words: ['the', 'what', 'is', 'i', 'how', 'a', 'to', 'in', 'do', 'of']
Least frequent words: ['intrinsic', 'painters', 'sculptors', 'extrinsic', 'textplus', 'echad', 'nordette', 'unmined', 'extremist', 'miller']


##### Convert Each Question to a Word2Vec Vector

In [90]:
def question_to_vec(question, model):
    """Convert a question to a Word2Vec vector by averaging the word vectors of the words in the question."""
    words = question.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vecs, axis=0)

# Convert questions to vectors
q1_vecs = ques_df['question1'].apply(lambda x: question_to_vec(x, w2v_model)).tolist()
q2_vecs = ques_df['question2'].apply(lambda x: question_to_vec(x, w2v_model)).tolist()

# Convert lists to numpy arrays
q1_arr = np.array(q1_vecs)
q2_arr = np.array(q2_vecs)


In [91]:
question_to_vec('what is the best marketing automation tool for small and mid size companies', w2v_model)

array([-0.19402538,  0.20130633, -0.0638341 , -0.00206332,  0.067021  ,
       -0.40539742,  0.17022517,  0.51233864, -0.5064507 , -0.47695494,
       -0.00222158, -0.33263737,  0.06848799,  0.28402263,  0.2143204 ,
       -0.0269644 ,  0.5103681 , -0.03441722, -0.14111075, -0.71094453,
        0.37239867, -0.03342674,  0.3818825 , -0.15446363, -0.02501704,
       -0.00203133, -0.34472755,  0.22668831, -0.22965494,  0.08319286,
        0.39303413, -0.13063842,  0.15004864, -0.29182905, -0.23598531,
        0.16670491,  0.1303893 , -0.03124342,  0.03250251, -0.37833044,
        0.06841131, -0.19769533, -0.07268234,  0.06422836,  0.178999  ,
        0.09214376, -0.18455346,  0.00681496, -0.02858896,  0.21786411,
       -0.00296471, -0.25546172, -0.10051799, -0.09290671,  0.04570218,
        0.023241  ,  0.20658064,  0.06691936, -0.2017257 ,  0.18330806,
        0.03882714,  0.05053363,  0.26449183,  0.07368442, -0.14838162,
        0.45490107,  0.09560488,  0.2692026 , -0.24034096,  0.31

In [92]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(20000, 200)

In [93]:
temp_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
345547,-0.255887,0.314694,0.032837,-0.182665,0.193300,-0.279271,0.279913,0.339852,-0.291964,-0.267468,...,0.201206,-0.031969,0.174528,-0.174372,0.458809,-0.040249,0.095122,-0.101609,0.074444,0.080679
211651,-0.210845,0.071775,0.067086,0.037001,0.054097,-0.302843,0.321381,0.508352,-0.385181,-0.343857,...,0.031538,0.063040,0.106549,-0.003870,0.254830,-0.023170,-0.018673,-0.110230,0.144062,0.025412
79862,-0.498118,0.472697,-0.015226,-0.034881,0.238083,-0.285220,0.380635,0.596580,-0.401227,-0.250712,...,0.238595,0.087594,0.332430,-0.141916,0.443235,0.090120,0.029551,-0.074099,0.045919,-0.126421
323663,-0.212362,0.283459,0.023209,-0.079459,0.049691,-0.314607,0.242436,0.462960,-0.554302,-0.223306,...,0.195527,0.098054,0.169658,-0.205375,0.317011,0.014032,-0.010993,-0.088621,0.079413,-0.103121
27254,-0.223472,0.371620,-0.043484,-0.006953,0.077976,-0.344253,0.341409,0.510079,-0.337220,-0.275493,...,0.096069,0.027933,0.202608,-0.101617,0.574283,-0.027037,0.150740,-0.064408,0.082177,-0.236050


In [94]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head(2)

(20000, 223)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,90,91,92,93,94,95,96,97,98,99
398782,1,75,76,13,13,12,26,0.46,0.874989,0.874989,...,0.192535,0.056285,0.061188,-0.037474,0.359546,0.075569,0.067408,-0.241511,0.067965,-0.108045
115086,0,48,56,13,16,8,24,0.33,0.666644,0.499988,...,0.192576,0.009611,0.148275,-0.138882,0.477841,0.074660,0.028325,-0.132318,-0.001467,-0.215239


In [95]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7905

In [97]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.78675

##### Same approach without stopwords

In [99]:
from nltk.corpus import stopwords
stopwords.words('english')# all stop words in english
def remove_stopwords(text):
    new_text = []   
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

ques_df['question1'] = ques_df['question1'].apply(remove_stopwords)
ques_df['question2'] = ques_df['question2'].apply(remove_stopwords)

In [100]:
ques_df.head(2)

,question1,question2
398782,best marketing automation tool small mid ...,best marketing automation tool small mid ...
115086,poor want invest,quite poor want rich


In [101]:
from gensim.models import Word2Vec
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])
tokenized_questions = [q.split() for q in questions]
#training skip-gram model 
w2v_model = Word2Vec(tokenized_questions, vector_size=100, window=5, min_count=1,sg=1) 

In [102]:
y = w2v_model.wv.index_to_key  # fetches you the words corresponding to vector representation
y

['best',
 'get',
 'india',
 'people',
 'would',
 'good',
 'like',
 'one',
 'make',
 'quora',
 'way',
 'life',
 'money',
 'time',
 'know',
 'someone',
 'new',
 'difference',
 'learn',
 'find',
 'think',
 'much',
 'use',
 'many',
 'work',
 'indian',
 'trump',
 'ever',
 'without',
 'first',
 'start',
 'mean',
 'world',
 'online',
 'become',
 '2',
 'year',
 'want',
 'better',
 'could',
 'love',
 'us',
 'job',
 'feel',
 'possible',
 'go',
 'take',
 'day',
 'english',
 '2016',
 'really',
 'things',
 '1k',
 '500',
 'weight',
 '3',
 'person',
 'used',
 '1',
 'notes',
 'phone',
 'ways',
 'engineering',
 'donald',
 'different',
 'using',
 'buy',
 'google',
 'number',
 'stop',
 'long',
 'old',
 'sex',
 'improve',
 'need',
 '5',
 'going',
 'books',
 'lose',
 'black',
 'app',
 'girl',
 'facebook',
 'questions',
 'language',
 'business',
 'years',
 'account',
 'thing',
 'women',
 'question',
 'movie',
 'change',
 'war',
 'book',
 'happen',
 'android',
 'see',
 'company',
 'president',
 'real',
 'fre

In [103]:
def question_to_vec(question, model):
    """Convert a question to a Word2Vec vector by averaging the word vectors of the words in the question."""
    words = question.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vecs, axis=0)

# Convert questions to vectors
q1_vecs = ques_df['question1'].apply(lambda x: question_to_vec(x, w2v_model)).tolist()
q2_vecs = ques_df['question2'].apply(lambda x: question_to_vec(x, w2v_model)).tolist()

# Convert lists to numpy arrays
q1_arr = np.array(q1_vecs)
q2_arr = np.array(q2_vecs)


In [104]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(20000, 200)

In [106]:
new_final_df = new_df.drop(columns=['id','qid1','qid2','question1','question2'])
print(new_final_df.shape)

(20000, 23)


In [107]:
new_final_df = pd.concat([new_final_df, temp_df], axis=1)
print(new_final_df.shape)
new_final_df.head(2)

(20000, 223)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,90,91,92,93,94,95,96,97,98,99
398782,1,75,76,13,13,12,26,0.46,0.874989,0.874989,...,0.216665,0.036384,0.209796,0.063682,0.566054,0.210279,0.180154,-0.298616,0.15325,0.054824
115086,0,48,56,13,16,8,24,0.33,0.666644,0.499988,...,0.189762,0.105578,0.077624,0.030823,0.551057,0.275875,0.211552,-0.104474,0.07467,0.134825


In [108]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(new_final_df.iloc[:,1:].values,new_final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [109]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.78525

In [110]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.79025

In [111]:
from sklearn.metrics import confusion_matrix
# for random forest model
confusion_matrix(y_test,y_pred)

array([[2131,  397],
       [ 462, 1010]], dtype=int64)

In [112]:
# for xgboost model
confusion_matrix(y_test,y_pred1)

array([[2120,  408],
       [ 431, 1041]], dtype=int64)

##### Use LSTM to train the model

In [125]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [126]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(len(w2v_model.wv.key_to_index)+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='sigmoid')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid')) 

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\saras\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\saras\AppData\Local\Temp\ipykernel_13040\733139925.py", line 5, in <module>
    model.add(Bidirectional(LSTM(32, activation='sigmoid')))
  File "c:\Users\saras\steffysara\VS_Code_Projects\Comment_Toxicity_Classifier\venv\lib\site-packages\tensorflow\python\training\tracking\base.py", line 522, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "c:\Users\saras\steffysara\VS_Code_Projects\Comment_Toxicity_Classifier\venv\lib\site-packages\tensorflow\python\keras\engine\sequential.py", line 228, in add
    output_tensor = layer(self.outputs[0])
  File "c:\Users\saras\steffysara\VS_Code_Projects\Comment_Toxicity_Classifier\venv\lib\site-packages\tensorflow\python\keras\layers\wrappers.py", line 585, in __call__
    return super(Bidirectional, 

In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam',metrics=['accuracy'])
model.summary()

In [124]:
np.object = object 
np.bool = bool 
np.int = int 
np.float = float 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional

# Define the maximum sequence length (after padding)
max_seq_length = 100  # Set based on your data

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(w2v_model.wv.key_to_index), output_dim=w2v_model.vector_size, input_length=max_seq_length))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\saras\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\saras\AppData\Local\Temp\ipykernel_13040\3309025809.py", line 14, in <module>
    model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
  File "c:\Users\saras\steffysara\VS_Code_Projects\Comment_Toxicity_Classifier\venv\lib\site-packages\tensorflow\python\training\tracking\base.py", line 522, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "c:\Users\saras\steffysara\VS_Code_Projects\Comment_Toxicity_Classifier\venv\lib\site-packages\tensorflow\python\keras\engine\sequential.py", line 228, in add
    output_tensor = layer(self.outputs[0])
  File "c:\Users\saras\steffysara\VS_Code_Projects\Comment_Toxicity_Classifier\venv\lib\site-packages\tensorflow\python\keras\layers\recurrent.py", line 668, in __call__
    return super(RNN, s

In [115]:
w2v_model.vector_size

100